In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

dictlist = []
totalSec = []

page = requests.get('https://www.senate.ca.gov/senators')
soup = BeautifulSoup(page.content, 'html.parser')

sectionOdd = soup.find_all("tr", {"class" : "odd"})
sectionEven = soup.find_all("tr", {"class" : "even"})

target = soup.find_all("td", {"class" : "views-field views-field-field-member-lname-sort active"})

# sum the odd columns and even columns together
for i in range(max(len(sectionOdd) ,len(sectionEven))):
    totalSec.append(sectionOdd[i])
    totalSec.append(sectionEven[i])

# get district and party from everyone
for sec in totalSec:
    
    data_object = []
    td = sec.find_all("td")

    # assign name and twitter link to the object
    res = td[0].find('a', href=True)
    data_object.append(res.text) # name

    p = requests.get(res['href'])
    new_page = BeautifulSoup(p.content, 'html.parser')
    
    t = new_page.find('div', {'id' : 'block-connectwithus'})
    if t is not None:
        t = t.find_all('a', href=True)
        data_object.append(t[1]['href']) # twitter link

        result = t[1]['href'].split('/')[3]
        result = result.replace('@', '')
        data_object.append(result) # twitter handle
    else:
        data_object.append("")
        data_object.append("")

    dist = td[1].text # District 
    par = td[2].text # Party
    dist = dist.replace("\n", "")
    dist = dist.strip() 
    par = par.replace("\n", "")
    par = par.strip()
    data_object.append(dist) # district
    data_object.append(par) # party

    dictlist.append(data_object)

df = pd.DataFrame(dictlist, columns=['name', 'twitter link', 'twitter handle',
         'district', 'party'])

df.to_csv('assemblyMembers/assemblyMembers_tweets.csv',index=False)

df.head()